In [1]:
from utils import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()
llm_config = {"model": "gpt-3.5-turbo"}

In [2]:
from autogen import ConversableAgent

c:\Users\kshantam\Time series forecasting\Agentic AI\env\Lib\site-packages\flaml\__init__.py:20: UserWarning: flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.
  warnings.warn("flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.")


### Creating the needed agents

In [3]:
onboarding_personal_information_agent = ConversableAgent(
    name="Onboarding Personal Information Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's name and location.
    Do not ask for other information. Return 'TERMINATE' 
    when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

[autogen.oai.client: 01-13 11:48:36] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [4]:
onboarding_topic_preference_agent = ConversableAgent(
    name="Onboarding Topic preference Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's preferences on news topics.
    Do not ask for other information.
    Return 'TERMINATE' when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

[autogen.oai.client: 01-13 11:48:37] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [5]:
customer_engagement_agent = ConversableAgent(
    name="Customer Engagement Agent",
    system_message='''You are a helpful customer service agent
    here to provide fun for the customer based on the user's
    personal information and topic preferences.
    This could include fun facts, jokes, or interesting stories.
    Make sure to make it engaging and fun!
    Return 'TERMINATE' when you are done.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

[autogen.oai.client: 01-13 11:48:37] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [6]:
customer_proxy_agent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

### Creating tasks

In [7]:
chats = [
    {
        "sender": onboarding_personal_information_agent,
        "recipient": customer_proxy_agent,
        "message": 
            "Hello, I'm here to help you get started with our product."
            "Could you tell me your name and location?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Return the customer information "
                             "into as JSON object only: "
                             "{'name': '', 'location': ''}",
        },
        "max_turns": 2,
        "clear_history" : True
    },
    {
        "sender": onboarding_topic_preference_agent,
        "recipient": customer_proxy_agent,
        "message": 
                "Great! Could you tell me what topics you are "
                "interested in reading about?",
        "summary_method": "reflection_with_llm",
        "max_turns": 1,
        "clear_history" : False
    },
    {
        "sender": customer_proxy_agent,
        "recipient": customer_engagement_agent,
        "message": "Let's find something fun to read.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
    },
]

### Start the onboarding process

In [8]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
Onboarding Personal Information Agent (to customer_proxy_agent):

Hello, I'm here to help you get started with our product.Could you tell me your name and location?

--------------------------------------------------------------------------------


c:\Users\kshantam\Time series forecasting\Agentic AI\env\Lib\site-packages\autogen\agentchat\chat.py:47: UserWarning: Repetitive recipients detected: The chat history will be cleared by default if a recipient appears more than once. To retain the chat history, please set 'clear_history=False' in the configuration of the repeating agent.
  warnings.warn(


customer_proxy_agent (to Onboarding Personal Information Agent):

Shantam

--------------------------------------------------------------------------------
Onboarding Personal Information Agent (to customer_proxy_agent):

Thank you for providing your name. Could you also share your location with me?

--------------------------------------------------------------------------------
customer_proxy_agent (to Onboarding Personal Information Agent):

Delhi

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Onboarding Topic preference Agent (to customer_proxy_agent):

Great! Could you tell me what topics you are interested in reading about?
Context: 
{
  "name": "Shantam",
  "location": "Delhi"
}

--------------------------------------------------------------------------------


### Print out the summary and cost

In [9]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

{
  "name": "Shantam",
  "location": "Delhi"
}


Shantam from Delhi is interested in reading about agentic AI.


Shantam from Delhi is interested in reading about agentic AI, which refers to AI systems that can act autonomously on behalf of a user. Agentic AI is becoming more prevalent in our everyday lives, revolutionizing how we interact with technology.




In [10]:
for chat_result in chat_results:
    print(chat_result.cost)
    print("\n")

{'usage_including_cached_inference': {'total_cost': 0.00014850000000000003, 'gpt-3.5-turbo-0125': {'cost': 0.00014850000000000003, 'prompt_tokens': 186, 'completion_tokens': 37, 'total_tokens': 223}}, 'usage_excluding_cached_inference': {'total_cost': 0.00014850000000000003, 'gpt-3.5-turbo-0125': {'cost': 0.00014850000000000003, 'prompt_tokens': 186, 'completion_tokens': 37, 'total_tokens': 223}}}


{'usage_including_cached_inference': {'total_cost': 5.8e-05, 'gpt-3.5-turbo-0125': {'cost': 5.8e-05, 'prompt_tokens': 71, 'completion_tokens': 15, 'total_tokens': 86}}, 'usage_excluding_cached_inference': {'total_cost': 5.8e-05, 'gpt-3.5-turbo-0125': {'cost': 5.8e-05, 'prompt_tokens': 71, 'completion_tokens': 15, 'total_tokens': 86}}}


{'usage_including_cached_inference': {'total_cost': 0.000371, 'gpt-3.5-turbo-0125': {'cost': 0.000371, 'prompt_tokens': 292, 'completion_tokens': 150, 'total_tokens': 442}}, 'usage_excluding_cached_inference': {'total_cost': 0.000371, 'gpt-3.5-turbo-0125': {